In [2]:
import recs
from langchain_community.document_loaders import TextLoader #convert text to format compatible
from langchain_text_splitters import CharacterTextSplitter #split text
from langchain_openai import OpenAIEmbeddings #convert text in embeddings
from langchain_chroma import Chroma #vector database
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter


In [3]:
from dotenv import load_dotenv #load api_keys

load_dotenv()

True

In [4]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")


In [5]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,titles_and_subtitles,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that:Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [6]:
books["tagged_description"].sample(5)

1439    9780345251220 The first novel in the series, "...
4455    9781405201711 Jo, Bessie and Fanny move to the...
4648    9781565849426 Parenti presents a story of popu...
869     9780143039495 The first translation of the Jap...
4008    9780812975932 Seventeen-year-old Manhattan soc...
Name: tagged_description, dtype: object

In [7]:
books["tagged_description"].to_csv(
    "tagged_description.txt",
    index=False,
    header=False
)

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# Carregar o arquivo com a codificação correta
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()

# Dividir os textos por quebra de linha
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)


Created a chunk of size 1170, which is longer than the specified 0
Created a chunk of size 1216, which is longer than the specified 0
Created a chunk of size 375, which is longer than the specified 0
Created a chunk of size 311, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 484, which is longer than the specified 0
Created a chunk of size 962, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 845, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1090, which is longer than the specified 0
Created a chunk of size 1191, which is longer than the specified 0
Created a chunk of size 306, which is longer than the specified 0
Create

In [9]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [10]:
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings())

In [11]:
query = "a book to teach children about nature"
doc = db_books.similarity_search(query, k=10)
doc

[Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child\'s drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780786808397 Introduce your baby to birds, cats, dogs, and babies through fine art

In [12]:
isbn = int(doc[0].page_content.strip('"').split()[0])
books[books["isbn13"] == isbn]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,titles_and_subtitles,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [13]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 20,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)

retrieve_semantic_recommendations("A book about economy")


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,titles_and_subtitles,tagged_description
444,9780071348270,0071348271,Mission Possible,Ken Blanchard;Terry Waghorn,Business & Economics,http://books.google.com/books/content?id=ofjQm...,Explains how to improve one's existing organiz...,1999.0,3.42,242.0,0.0,Mission Possible:Becoming a World Class Organi...,9780071348270 Explains how to improve one's ex...
483,9780099428633,0099428636,How to Travel with a Salmon and Other Essays,Umberto Eco,Essays,http://books.google.com/books/content?id=-SLft...,This witty and irreverent collection of essays...,2001.0,3.85,229.0,106.0,How to Travel with a Salmon and Other Essays,9780099428633 This witty and irreverent collec...
570,9780140143454,0140143459,Liar's Poker,Michael M. Lewis,Biography & Autobiography,http://books.google.com/books/content?id=eboqx...,The author recounts his experiences on the luc...,1990.0,4.15,256.0,22710.0,Liar's Poker:Rising Through the Wreckage on Wa...,9780140143454 The author recounts his experien...
586,9780140187076,0140187073,J R,William Gaddis,Fiction,http://books.google.com/books/content?id=DLSmn...,"At the center of this hugely comic tale of ""fr...",1993.0,4.28,752.0,2027.0,J R,9780140187076 At the center of this hugely com...
626,9780140285970,0140285970,Shadows and Wind,Robert Templer,History,http://books.google.com/books/content?id=CFiaQ...,"Examines the problems facing modern Vietnam, a...",1998.0,3.48,400.0,101.0,Shadows and Wind:A View of Modern Vietnam,9780140285970 Examines the problems facing mod...
844,9780143036081,0143036084,The United States of Europe,T. R. Reid,History,http://books.google.com/books/content?id=Yra-G...,Now comprising 25 nations and 450 million citi...,2005.0,3.62,320.0,681.0,The United States of Europe:The New Superpower...,9780143036081 Now comprising 25 nations and 45...
865,9780143038825,0143038826,The White Man's Burden,William Easterly,Business & Economics,http://books.google.com/books/content?id=8SayN...,Argues that western foreign aid efforts have d...,2006.0,3.83,436.0,4317.0,The White Man's Burden:Why the West's Efforts ...,9780143038825 Argues that western foreign aid ...
870,9780143039501,0143039504,The Complete Novels,Jane Austen,Fiction,http://books.google.com/books/content?id=a6Lok...,A definitive compilation of the author's seven...,2006.0,4.55,1278.0,417.0,The Complete Novels,9780143039501 A definitive compilation of the ...
1055,9780226675763,0226675769,"Antitrust Law, Second Edition",Richard A. Posner,Law,http://books.google.com/books/content?id=wvkym...,When it was first published a quarter of a cen...,2001.0,4.00,304.0,33.0,"Antitrust Law, Second Edition",9780226675763 When it was first published a qu...
1097,9780262611336,0262611333,Pop Internationalism,Paul R. Krugman,Business & Economics,http://books.google.com/books/content?id=17Yrn...,"""Pop internationalists""--people who speak impr...",1997.0,3.67,221.0,307.0,Pop Internationalism,"9780262611336 ""Pop internationalists""--people ..."


In [14]:
retrieve_semantic_recommendations("a book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,titles_and_subtitles,tagged_description
31,9780007105045,0007105045,Tree and Leaf,John Ronald Reuel Tolkien,Literary Collections,http://books.google.com/books/content?id=aPb_A...,"""The two works 'On fairy-stories' and 'Leaf by...",2001.0,4.09,176.0,2245.0,Tree and Leaf:The Homecoming of Beorhtnoth : B...,"9780007105045 ""The two works 'On fairy-stories..."
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,9780007151240 It should be the most natural th...
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer:A Novel,9780060959036 Barbara Kingsolver's fifth novel...
400,9780062700254,0062700251,Bulfinch's Mythology,Richard P. Martin,Reference,http://books.google.com/books/content?id=eev4u...,A beautiful gift edition of Thomas Bulfinch's ...,1991.0,4.10,768.0,64.0,"Bulfinch's Mythology:The Age of the Fable, The...",9780062700254 A beautiful gift edition of Thom...
429,9780064434980,0064434982,The Deer in the Wood,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=V7YDW...,Even the youngest child can enjoy a special ad...,1999.0,4.17,32.0,302.0,The Deer in the Wood,9780064434980 Even the youngest child can enjo...
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208 First published more than three ...
692,9780140448009,0140448004,Three Tales,Gustave Flaubert;Roger Whitehouse;Geoffrey Wall,Fiction,http://books.google.com/books/content?id=XFzga...,Features short fiction by the French naturalis...,2005.0,3.71,110.0,3050.0,Three Tales,9780140448009 Features short fiction by the Fr...
707,9780140568196,0140568190,The Giraffe and the Pelly and Me,Roald Dahl;Quentin Blake,Candy,http://books.google.com/books/content?id=J7FdI...,"A Dahl story in which the giraffe, the pelican...",2001.0,3.81,32.0,16265.0,The Giraffe and the Pelly and Me,9780140568196 A Dahl story in which the giraff...
711,9780140621624,0140621628,The Railway Children,E. Nesbit,Fiction,http://books.google.com/books/content?id=fFesd...,"When their father is sent away to prison, thre...",1995.0,4.00,212.0,178.0,The Railway Children,9780140621624 When their father is sent away t...
763,9780141186078,0141186070,The Log from the Sea of Cortez,John Steinbeck,Biography & Autobiography,http://books.google.com/books/content?id=9CrIf...,This light-hearted journal tells of John Stein...,2001.0,3.84,288.0,3226.0,The Log from the Sea of Cortez,9780141186078 This light-hearted journal tells...
